In [ ]:
!pip install ultralytics


# Prepare dataset ready

In [ ]:
import zipfile
import os
import pandas as pd


# Config to train model

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
model = YOLO("yolov8n.pt")
model = YOLO("yolo11n.pt")

In [ ]:
!cp -r /kaggle/input/came-car /kaggle/working/

In [ ]:
batch_size = [2, 4, 8]
optimizer = ['AdamW','SGD']
ep = 100
learning_rates = [0.01, 1e-3, 1e-5]
results = {}  # Store results

for lr in learning_rates:
    for bs in batch_size:
        for opt in optimizer:
            # Train the model
            #model = YOLO('/kaggle/working/yolov8n.pt')  # Reset model
            #model = YOLO('/kaggle/working/yolo11n.pt')  # Reset model

            model.train(
                data='/kaggle/working/came-car/Data_329/data.yaml',
                epochs=ep,
                imgsz=640,
                batch=bs,
                optimizer=opt,
                lr0=lr,
                cache='/kaggle/working/cache',
                name=f'bs{bs}_opt{opt}_ep{ep}'  # Name each run
            )
            
            # Đọc dữ liệu từ results.csv
            results_data = model.trainer.read_results_csv()  # Đổi tên thành results_data

            # Chuyển đổi thành DataFrame (nếu cần)
            df = pd.DataFrame(results_data)

            # Lưu DataFrame vào file CSV
            df.to_csv(f'/kaggle/working/training_results_{bs}_{opt}_{ep}_{lr}.csv', index=False) 
            
            # Evaluate model and save results
            results[f'bs{bs}_opt{opt}_ep{ep}_lr{lr}'] = model.val()
